In [54]:
import numpy as np
import pandas as pd
import datetime

In [52]:
def getEveryDay(begin_date,end_date):
    date_list = []
    date_flag = []
    begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        if begin_date.strftime("%m-%d") >= '11-01' and begin_date.strftime("%m-%d") <= '11-11' or \
                begin_date.strftime("%m-%d") >= '12-01' and begin_date.strftime("%m-%d") <= '12-12' or \
                   begin_date.strftime("%m-%d") >= '10-01' and begin_date.strftime("%m-%d") <= '10-07' :
            date_flag.append(2)
        elif begin_date.strftime("%w") == '0' or begin_date.strftime("%w") == '6':
            date_flag.append(1)
        else:
            date_flag.append(0)
        begin_date += datetime.timedelta(days=1)
    return date_list, date_flag

def spatial_partition(purchase_data, cart_data, cate_list, area_flag, date_list):
    for area in area_flag:
        area_purchase = []
        for date in date_list:
            date_purchase = []
            for cate in cate_list:
                entry = purchase_data[(purchase_data['日期'] == date) & (purchase_data['市'] == area) & (purchase_data['一级类目'] == cate)]['数量']
                if len(entry) == 0:
                    date_purchase.append(0)
                else:
                    date_purchase.append(int(entry))
            area_purchase.append(date_purchase)
            print(area, date)
        np.save('../data/spatial/purchase/' + area + '.npy', np.array(area_purchase))

def temporal_partition(purchase_data, cart_data, cate_list, date_flag, date_list):
    pass

In [3]:
data_dir = '../data/'
begin_date, end_date = '2015-01-01', '2019-12-18'
begin_year, end_year = 2015, 2020
categories_purchase = []
categories_cart = []
area_purchase = []
area_cart = []
dates, date_flag = getEveryDay(begin_date, end_date)

In [4]:
for year in range(begin_year, end_year):
    purchase_data = pd.read_csv(data_dir + 'purchase-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    # select categoties of products in purchase data
    categories = purchase_data.groupby('一级类目').nunique()
    categories_purchase.append(set(categories['一级类目'].index.tolist()))
    # select all area in purchase data
    areas = purchase_data.groupby('市').nunique()
    area_purchase.append(set(areas['市'].index.tolist()))

    cart_data = pd.read_csv(data_dir + 'shoppingcart-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    # select categoties of products in shopping cart data
    categories = cart_data.groupby('一级类目').nunique()
    categories_cart.append(set(categories['一级类目'].index.tolist()))
    # select all area in shopping cart data
    areas = cart_data.groupby('市').nunique()
    area_cart.append(set(areas['市'].index.tolist()))

categories_purchase = categories_purchase[0] & categories_purchase[1] & categories_purchase[2] & categories_purchase[3] & categories_purchase[4]
categories_purchase = categories_purchase - set(['\\N', '测试分类'])
area_purchase = area_purchase[0] & area_purchase[1] & area_purchase[2] & area_purchase[3] & area_purchase[4]

categories_cart = categories_cart[0] & categories_cart[1] & categories_cart[2] & categories_cart[3] & categories_cart[4]
categories_cart = categories_cart - set(['\\N', '测试分类'])
area_cart = area_cart[0] & area_cart[1] & area_cart[2] & area_cart[3] & area_cart[4]

categories = list(categories_purchase & categories_cart)
areas = list(area_purchase & area_cart)

print(categories, len(categories))
print(areas, len(areas))


['厨具', '服饰内衣', '家用电器', '鞋靴', '家装建材', '宠物生活', '电脑、办公', '图书', '家具', '手机通讯', '玩具乐器', '本地生活/旅游出行', '钟表', '酒类', '影视', '医药保健', '农资园艺', '美妆护肤', '整车', '文娱', '礼品', '珠宝首饰', '食品饮料', '家居日用', '汽车用品', '运动户外', '生鲜', '音乐', '母婴', '数码'] 30
['怀柔区', '大兴区', '昌平区', '通州区', '石景山区', '延庆区', '顺义区', '西城区', '平谷区', '宣武区', '东城区', '丰台区', '朝阳区', '门头沟', '崇文区', '房山区', '密云区', '海淀区'] 18


In [51]:
for year in range(begin_year, end_year):
    purchase_data = pd.read_csv(data_dir + 'purchase-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    purchase_data = purchase_data.drop(['省', '区'], axis=1)
    purchase_data['一级类目'] = purchase_data['一级类目'].str.strip()
    purchase_data = purchase_data.groupby(['日期', '市', '一级类目']).sum().reset_index()
    
    cart_data = pd.read_csv(data_dir + 'shoppingcart-' + str(year) + '.csv', sep = '\t', error_bad_lines=False)
    cart_data = cart_data.drop(['省', '区'], axis=1)
    cart_data['一级类目'] = cart_data['一级类目'].str.strip()
    cart_data = cart_data.groupby(['日期', '市', '一级类目']).sum().reset_index() 
    print(len(purchase_data), len(cart_data))
    
    if year == begin_year:
        purchase = purchase_data
        cart = cart_data
    else:
        purchase = pd.concat([purchase, purchase_data])
        cart = pd.concat([cart, cart_data])
print(len(purchase), len(cart))

228973 108709
232995 186203
250604 221889
263918 265114
309433 260485
1285923 1042400


In [55]:
spatial_partition(purchase_data, cart_data, categories, areas, dates)

怀柔区 2015-01-01
怀柔区 2015-01-02
怀柔区 2015-01-03
怀柔区 2015-01-04
怀柔区 2015-01-05
怀柔区 2015-01-06
怀柔区 2015-01-07
怀柔区 2015-01-08
怀柔区 2015-01-09
怀柔区 2015-01-10
怀柔区 2015-01-11
怀柔区 2015-01-12
怀柔区 2015-01-13
怀柔区 2015-01-14
怀柔区 2015-01-15
怀柔区 2015-01-16
怀柔区 2015-01-17
怀柔区 2015-01-18
怀柔区 2015-01-19
怀柔区 2015-01-20
怀柔区 2015-01-21
怀柔区 2015-01-22
怀柔区 2015-01-23
怀柔区 2015-01-24
怀柔区 2015-01-25
怀柔区 2015-01-26
怀柔区 2015-01-27
怀柔区 2015-01-28
怀柔区 2015-01-29
怀柔区 2015-01-30
怀柔区 2015-01-31
怀柔区 2015-02-01
怀柔区 2015-02-02
怀柔区 2015-02-03
怀柔区 2015-02-04
怀柔区 2015-02-05
怀柔区 2015-02-06
怀柔区 2015-02-07
怀柔区 2015-02-08
怀柔区 2015-02-09
怀柔区 2015-02-10
怀柔区 2015-02-11
怀柔区 2015-02-12
怀柔区 2015-02-13
怀柔区 2015-02-14
怀柔区 2015-02-15
怀柔区 2015-02-16
怀柔区 2015-02-17
怀柔区 2015-02-18
怀柔区 2015-02-19
怀柔区 2015-02-20
怀柔区 2015-02-21
怀柔区 2015-02-22
怀柔区 2015-02-23
怀柔区 2015-02-24
怀柔区 2015-02-25
怀柔区 2015-02-26
怀柔区 2015-02-27
怀柔区 2015-02-28
怀柔区 2015-03-01
怀柔区 2015-03-02
怀柔区 2015-03-03
怀柔区 2015-03-04
怀柔区 2015-03-05
怀柔区 2015-03-06
怀柔区 2015-03-07
怀柔区 2015-0

怀柔区 2016-07-01
怀柔区 2016-07-02
怀柔区 2016-07-03
怀柔区 2016-07-04
怀柔区 2016-07-05
怀柔区 2016-07-06
怀柔区 2016-07-07
怀柔区 2016-07-08
怀柔区 2016-07-09
怀柔区 2016-07-10
怀柔区 2016-07-11
怀柔区 2016-07-12
怀柔区 2016-07-13
怀柔区 2016-07-14
怀柔区 2016-07-15
怀柔区 2016-07-16
怀柔区 2016-07-17
怀柔区 2016-07-18
怀柔区 2016-07-19
怀柔区 2016-07-20
怀柔区 2016-07-21
怀柔区 2016-07-22
怀柔区 2016-07-23
怀柔区 2016-07-24
怀柔区 2016-07-25
怀柔区 2016-07-26
怀柔区 2016-07-27
怀柔区 2016-07-28
怀柔区 2016-07-29
怀柔区 2016-07-30
怀柔区 2016-07-31
怀柔区 2016-08-01
怀柔区 2016-08-02
怀柔区 2016-08-03
怀柔区 2016-08-04
怀柔区 2016-08-05
怀柔区 2016-08-06
怀柔区 2016-08-07
怀柔区 2016-08-08
怀柔区 2016-08-09
怀柔区 2016-08-10
怀柔区 2016-08-11
怀柔区 2016-08-12
怀柔区 2016-08-13
怀柔区 2016-08-14
怀柔区 2016-08-15
怀柔区 2016-08-16
怀柔区 2016-08-17
怀柔区 2016-08-18
怀柔区 2016-08-19
怀柔区 2016-08-20
怀柔区 2016-08-21
怀柔区 2016-08-22
怀柔区 2016-08-23
怀柔区 2016-08-24
怀柔区 2016-08-25
怀柔区 2016-08-26
怀柔区 2016-08-27
怀柔区 2016-08-28
怀柔区 2016-08-29
怀柔区 2016-08-30
怀柔区 2016-08-31
怀柔区 2016-09-01
怀柔区 2016-09-02
怀柔区 2016-09-03
怀柔区 2016-09-04
怀柔区 2016-0

怀柔区 2017-12-30
怀柔区 2017-12-31
怀柔区 2018-01-01
怀柔区 2018-01-02
怀柔区 2018-01-03
怀柔区 2018-01-04
怀柔区 2018-01-05
怀柔区 2018-01-06
怀柔区 2018-01-07
怀柔区 2018-01-08
怀柔区 2018-01-09
怀柔区 2018-01-10
怀柔区 2018-01-11
怀柔区 2018-01-12
怀柔区 2018-01-13
怀柔区 2018-01-14
怀柔区 2018-01-15
怀柔区 2018-01-16
怀柔区 2018-01-17
怀柔区 2018-01-18
怀柔区 2018-01-19
怀柔区 2018-01-20
怀柔区 2018-01-21
怀柔区 2018-01-22
怀柔区 2018-01-23
怀柔区 2018-01-24
怀柔区 2018-01-25
怀柔区 2018-01-26
怀柔区 2018-01-27
怀柔区 2018-01-28
怀柔区 2018-01-29
怀柔区 2018-01-30
怀柔区 2018-01-31
怀柔区 2018-02-01
怀柔区 2018-02-02
怀柔区 2018-02-03
怀柔区 2018-02-04
怀柔区 2018-02-05
怀柔区 2018-02-06
怀柔区 2018-02-07
怀柔区 2018-02-08
怀柔区 2018-02-09
怀柔区 2018-02-10
怀柔区 2018-02-11
怀柔区 2018-02-12
怀柔区 2018-02-13
怀柔区 2018-02-14
怀柔区 2018-02-15
怀柔区 2018-02-16
怀柔区 2018-02-17
怀柔区 2018-02-18
怀柔区 2018-02-19
怀柔区 2018-02-20
怀柔区 2018-02-21
怀柔区 2018-02-22
怀柔区 2018-02-23
怀柔区 2018-02-24
怀柔区 2018-02-25
怀柔区 2018-02-26
怀柔区 2018-02-27
怀柔区 2018-02-28
怀柔区 2018-03-01
怀柔区 2018-03-02
怀柔区 2018-03-03
怀柔区 2018-03-04
怀柔区 2018-03-05
怀柔区 2018-0

怀柔区 2019-06-30
怀柔区 2019-07-01
怀柔区 2019-07-02
怀柔区 2019-07-03
怀柔区 2019-07-04
怀柔区 2019-07-05
怀柔区 2019-07-06
怀柔区 2019-07-07
怀柔区 2019-07-08
怀柔区 2019-07-09
怀柔区 2019-07-10
怀柔区 2019-07-11
怀柔区 2019-07-12
怀柔区 2019-07-13
怀柔区 2019-07-14
怀柔区 2019-07-15
怀柔区 2019-07-16
怀柔区 2019-07-17
怀柔区 2019-07-18
怀柔区 2019-07-19
怀柔区 2019-07-20
怀柔区 2019-07-21
怀柔区 2019-07-22
怀柔区 2019-07-23
怀柔区 2019-07-24
怀柔区 2019-07-25
怀柔区 2019-07-26
怀柔区 2019-07-27
怀柔区 2019-07-28
怀柔区 2019-07-29
怀柔区 2019-07-30
怀柔区 2019-07-31
怀柔区 2019-08-01
怀柔区 2019-08-02
怀柔区 2019-08-03
怀柔区 2019-08-04
怀柔区 2019-08-05
怀柔区 2019-08-06
怀柔区 2019-08-07
怀柔区 2019-08-08
怀柔区 2019-08-09
怀柔区 2019-08-10
怀柔区 2019-08-11
怀柔区 2019-08-12
怀柔区 2019-08-13
怀柔区 2019-08-14
怀柔区 2019-08-15
怀柔区 2019-08-16
怀柔区 2019-08-17
怀柔区 2019-08-18
怀柔区 2019-08-19
怀柔区 2019-08-20
怀柔区 2019-08-21
怀柔区 2019-08-22
怀柔区 2019-08-23
怀柔区 2019-08-24
怀柔区 2019-08-25
怀柔区 2019-08-26
怀柔区 2019-08-27
怀柔区 2019-08-28
怀柔区 2019-08-29
怀柔区 2019-08-30
怀柔区 2019-08-31
怀柔区 2019-09-01
怀柔区 2019-09-02
怀柔区 2019-09-03
怀柔区 2019-0

大兴区 2016-01-11
大兴区 2016-01-12
大兴区 2016-01-13
大兴区 2016-01-14
大兴区 2016-01-15
大兴区 2016-01-16
大兴区 2016-01-17
大兴区 2016-01-18
大兴区 2016-01-19
大兴区 2016-01-20
大兴区 2016-01-21
大兴区 2016-01-22
大兴区 2016-01-23
大兴区 2016-01-24
大兴区 2016-01-25
大兴区 2016-01-26
大兴区 2016-01-27
大兴区 2016-01-28
大兴区 2016-01-29
大兴区 2016-01-30
大兴区 2016-01-31
大兴区 2016-02-01
大兴区 2016-02-02
大兴区 2016-02-03
大兴区 2016-02-04
大兴区 2016-02-05
大兴区 2016-02-06
大兴区 2016-02-07
大兴区 2016-02-08
大兴区 2016-02-09
大兴区 2016-02-10
大兴区 2016-02-11
大兴区 2016-02-12
大兴区 2016-02-13
大兴区 2016-02-14
大兴区 2016-02-15
大兴区 2016-02-16
大兴区 2016-02-17
大兴区 2016-02-18
大兴区 2016-02-19
大兴区 2016-02-20
大兴区 2016-02-21
大兴区 2016-02-22
大兴区 2016-02-23
大兴区 2016-02-24
大兴区 2016-02-25
大兴区 2016-02-26
大兴区 2016-02-27
大兴区 2016-02-28
大兴区 2016-02-29
大兴区 2016-03-01
大兴区 2016-03-02
大兴区 2016-03-03
大兴区 2016-03-04
大兴区 2016-03-05
大兴区 2016-03-06
大兴区 2016-03-07
大兴区 2016-03-08
大兴区 2016-03-09
大兴区 2016-03-10
大兴区 2016-03-11
大兴区 2016-03-12
大兴区 2016-03-13
大兴区 2016-03-14
大兴区 2016-03-15
大兴区 2016-03-16
大兴区 2016-0

...

密云区 2017-08-14
密云区 2017-08-15
密云区 2017-08-16
密云区 2017-08-17
密云区 2017-08-18
密云区 2017-08-19
密云区 2017-08-20
密云区 2017-08-21
密云区 2017-08-22
密云区 2017-08-23
密云区 2017-08-24
密云区 2017-08-25
密云区 2017-08-26
密云区 2017-08-27
密云区 2017-08-28
密云区 2017-08-29
密云区 2017-08-30
密云区 2017-08-31
密云区 2017-09-01
密云区 2017-09-02
密云区 2017-09-03
密云区 2017-09-04
密云区 2017-09-05
密云区 2017-09-06
密云区 2017-09-07
密云区 2017-09-08
密云区 2017-09-09
密云区 2017-09-10
密云区 2017-09-11
密云区 2017-09-12
密云区 2017-09-13
密云区 2017-09-14
密云区 2017-09-15
密云区 2017-09-16
密云区 2017-09-17
密云区 2017-09-18
密云区 2017-09-19
密云区 2017-09-20
密云区 2017-09-21
密云区 2017-09-22
密云区 2017-09-23
密云区 2017-09-24
密云区 2017-09-25
密云区 2017-09-26
密云区 2017-09-27
密云区 2017-09-28
密云区 2017-09-29
密云区 2017-09-30
密云区 2017-10-01
密云区 2017-10-02
密云区 2017-10-03
密云区 2017-10-04
密云区 2017-10-05
密云区 2017-10-06
密云区 2017-10-07
密云区 2017-10-08
密云区 2017-10-09
密云区 2017-10-10
密云区 2017-10-11
密云区 2017-10-12
密云区 2017-10-13
密云区 2017-10-14
密云区 2017-10-15
密云区 2017-10-16
密云区 2017-10-17
密云区 2017-10-18
密云区 2017-1

密云区 2019-02-12
密云区 2019-02-13
密云区 2019-02-14
密云区 2019-02-15
密云区 2019-02-16
密云区 2019-02-17
密云区 2019-02-18
密云区 2019-02-19
密云区 2019-02-20
密云区 2019-02-21
密云区 2019-02-22
密云区 2019-02-23
密云区 2019-02-24
密云区 2019-02-25
密云区 2019-02-26
密云区 2019-02-27
密云区 2019-02-28
密云区 2019-03-01
密云区 2019-03-02
密云区 2019-03-03
密云区 2019-03-04
密云区 2019-03-05
密云区 2019-03-06
密云区 2019-03-07
密云区 2019-03-08
密云区 2019-03-09
密云区 2019-03-10
密云区 2019-03-11
密云区 2019-03-12
密云区 2019-03-13
密云区 2019-03-14
密云区 2019-03-15
密云区 2019-03-16
密云区 2019-03-17
密云区 2019-03-18
密云区 2019-03-19
密云区 2019-03-20
密云区 2019-03-21
密云区 2019-03-22
密云区 2019-03-23
密云区 2019-03-24
密云区 2019-03-25
密云区 2019-03-26
密云区 2019-03-27
密云区 2019-03-28
密云区 2019-03-29
密云区 2019-03-30
密云区 2019-03-31
密云区 2019-04-01
密云区 2019-04-02
密云区 2019-04-03
密云区 2019-04-04
密云区 2019-04-05
密云区 2019-04-06
密云区 2019-04-07
密云区 2019-04-08
密云区 2019-04-09
密云区 2019-04-10
密云区 2019-04-11
密云区 2019-04-12
密云区 2019-04-13
密云区 2019-04-14
密云区 2019-04-15
密云区 2019-04-16
密云区 2019-04-17
密云区 2019-04-18
密云区 2019-0

海淀区 2015-08-26
海淀区 2015-08-27
海淀区 2015-08-28
海淀区 2015-08-29
海淀区 2015-08-30
海淀区 2015-08-31
海淀区 2015-09-01
海淀区 2015-09-02
海淀区 2015-09-03
海淀区 2015-09-04
海淀区 2015-09-05
海淀区 2015-09-06
海淀区 2015-09-07
海淀区 2015-09-08
海淀区 2015-09-09
海淀区 2015-09-10
海淀区 2015-09-11
海淀区 2015-09-12
海淀区 2015-09-13
海淀区 2015-09-14
海淀区 2015-09-15
海淀区 2015-09-16
海淀区 2015-09-17
海淀区 2015-09-18
海淀区 2015-09-19
海淀区 2015-09-20
海淀区 2015-09-21
海淀区 2015-09-22
海淀区 2015-09-23
海淀区 2015-09-24
海淀区 2015-09-25
海淀区 2015-09-26
海淀区 2015-09-27
海淀区 2015-09-28
海淀区 2015-09-29
海淀区 2015-09-30
海淀区 2015-10-01
海淀区 2015-10-02
海淀区 2015-10-03
海淀区 2015-10-04
海淀区 2015-10-05
海淀区 2015-10-06
海淀区 2015-10-07
海淀区 2015-10-08
海淀区 2015-10-09
海淀区 2015-10-10
海淀区 2015-10-11
海淀区 2015-10-12
海淀区 2015-10-13
海淀区 2015-10-14
海淀区 2015-10-15
海淀区 2015-10-16
海淀区 2015-10-17
海淀区 2015-10-18
海淀区 2015-10-19
海淀区 2015-10-20
海淀区 2015-10-21
海淀区 2015-10-22
海淀区 2015-10-23
海淀区 2015-10-24
海淀区 2015-10-25
海淀区 2015-10-26
海淀区 2015-10-27
海淀区 2015-10-28
海淀区 2015-10-29
海淀区 2015-10-30
海淀区 2015-1

海淀区 2017-02-23
海淀区 2017-02-24
海淀区 2017-02-25
海淀区 2017-02-26
海淀区 2017-02-27
海淀区 2017-02-28
海淀区 2017-03-01
海淀区 2017-03-02
海淀区 2017-03-03
海淀区 2017-03-04
海淀区 2017-03-05
海淀区 2017-03-06
海淀区 2017-03-07
海淀区 2017-03-08
海淀区 2017-03-09
海淀区 2017-03-10
海淀区 2017-03-11
海淀区 2017-03-12
海淀区 2017-03-13
海淀区 2017-03-14
海淀区 2017-03-15
海淀区 2017-03-16
海淀区 2017-03-17
海淀区 2017-03-18
海淀区 2017-03-19
海淀区 2017-03-20
海淀区 2017-03-21
海淀区 2017-03-22
海淀区 2017-03-23
海淀区 2017-03-24
海淀区 2017-03-25
海淀区 2017-03-26
海淀区 2017-03-27
海淀区 2017-03-28
海淀区 2017-03-29
海淀区 2017-03-30
海淀区 2017-03-31
海淀区 2017-04-01
海淀区 2017-04-02
海淀区 2017-04-03
海淀区 2017-04-04
海淀区 2017-04-05
海淀区 2017-04-06
海淀区 2017-04-07
海淀区 2017-04-08
海淀区 2017-04-09
海淀区 2017-04-10
海淀区 2017-04-11
海淀区 2017-04-12
海淀区 2017-04-13
海淀区 2017-04-14
海淀区 2017-04-15
海淀区 2017-04-16
海淀区 2017-04-17
海淀区 2017-04-18
海淀区 2017-04-19
海淀区 2017-04-20
海淀区 2017-04-21
海淀区 2017-04-22
海淀区 2017-04-23
海淀区 2017-04-24
海淀区 2017-04-25
海淀区 2017-04-26
海淀区 2017-04-27
海淀区 2017-04-28
海淀区 2017-04-29
海淀区 2017-0

海淀区 2018-08-24
海淀区 2018-08-25
海淀区 2018-08-26
海淀区 2018-08-27
海淀区 2018-08-28
海淀区 2018-08-29
海淀区 2018-08-30
海淀区 2018-08-31
海淀区 2018-09-01
海淀区 2018-09-02
海淀区 2018-09-03
海淀区 2018-09-04
海淀区 2018-09-05
海淀区 2018-09-06
海淀区 2018-09-07
海淀区 2018-09-08
海淀区 2018-09-09
海淀区 2018-09-10
海淀区 2018-09-11
海淀区 2018-09-12
海淀区 2018-09-13
海淀区 2018-09-14
海淀区 2018-09-15
海淀区 2018-09-16
海淀区 2018-09-17
海淀区 2018-09-18
海淀区 2018-09-19
海淀区 2018-09-20
海淀区 2018-09-21
海淀区 2018-09-22
海淀区 2018-09-23
海淀区 2018-09-24
海淀区 2018-09-25
海淀区 2018-09-26
海淀区 2018-09-27
海淀区 2018-09-28
海淀区 2018-09-29
海淀区 2018-09-30
海淀区 2018-10-01
海淀区 2018-10-02
海淀区 2018-10-03
海淀区 2018-10-04
海淀区 2018-10-05
海淀区 2018-10-06
海淀区 2018-10-07
海淀区 2018-10-08
海淀区 2018-10-09
海淀区 2018-10-10
海淀区 2018-10-11
海淀区 2018-10-12
海淀区 2018-10-13
海淀区 2018-10-14
海淀区 2018-10-15
海淀区 2018-10-16
海淀区 2018-10-17
海淀区 2018-10-18
海淀区 2018-10-19
海淀区 2018-10-20
海淀区 2018-10-21
海淀区 2018-10-22
海淀区 2018-10-23
海淀区 2018-10-24
海淀区 2018-10-25
海淀区 2018-10-26
海淀区 2018-10-27
海淀区 2018-10-28
海淀区 2018-1